# Slot Evaluation and Reporting

Author: [@abhinav](https://lepisma.xyz/)

Notebook for working on the slot evaluation problem for speech utterances for spoken dialog systems. We will cover the following in the session:

1. Differences between slots and entities
1. Standard slot-filling metrics
1. Issues with just Precision and Recall
1. MM, MF, NF
1. What to report?
1. How to report?
1. Action items

## 1. Slots and Entities

At the moment we use intent and slot formulation to _understand_ utterances. Note than an utterance here has the regular meaning and when we say we go from an utterance to a semantic parse, an intermediate ASR might or might not be involved.

In the current formulation, a **slot** maps to a semantic unit which works like an attribute for the intent. For example `arrival_city` is a slot that might be associated with `flight_booking` intent.

A **slot** is a high level construct which, while generalizable, is bound to the use case. Slot can support multiple _type_ of **entities** which finally are _filled_ in the slot (in the factored parsing model, where we first parse entities and then _fill_ slot). For example `arrival_city` can support a type `city`.

Since they are mostly portable set of tokens, **entities** are usually, not always, identifiable via a span in text. Here is one of the span parser's output which provides the span description as substring _range_:

```python
text = "Sunday morning love you"

entity = {
    "body": "Sunday morning",
    "range": {
        "start": 0,
        "end": 14
    },
    "score": None,
    "value": ...,
    "type": "datetime"
}
```

Our current way of working with ASR involves using n-best alternatives for each utterance. This means any span based entity parser also has to locate the span among the alternatives and that also shows up in the final parsing output.

There are many reasons why entities and slots should be looked at separately. While you can always _level up_ an entity, we try to keep entities at a syntactic level and slots at semantic level. This means that entities cover what the user _said_, slots cover what the user _meant_. Here are a few concerns that can help understand the differences better:

1. N-best noise
2. Speech disfluencies
3. Type disambiguation
4. Filling operators
5. Repairs and multi turn recovery


### 1.1. N-best noise

The user has uttered something but the ASR's noise, represented in the n-best list, is making us hard to understand what the user _said_. Here is an example where the user actually said `7 pm`:

```python
alternatives = [[
    {"transcript": "i was born at 7 pm"},
    {"transcript": "i was born at 11 pm"},
    {"transcript": "i was born at seven pm"}
]]
```

This is a problem at entity parser level. While the slot could also be equivalent to entity in this case, this won't be true always.


### 1.2. Speech disfluencies

Now consider disfluencies like repetitions and repairs. Suppose a person said `3 pm, no no, 5 pm` in response to a retaurant booking time query. In this case, it's okay, if designed, for a time entity parser to capture `3 pm` and `5 pm`. But the slot `booking_time` should capture `5 pm`. We tend to merge parsing for this, but it's a distinction to be aware of.


### 1.3. Type disambiguation

At times you might have two slots with same entity type. When the entity is captured, you will have to decide which slot it fits in. Usually this is not a problem across turns, but you might get this issue if the same turn contains both entities. Here is an example utterance from flight booking: "Bangalore to Delhi". There are two slots here, `arrival_city` and `departure_city`. Both might support entity type `city`. While parsing the above mentioned sentence, we can use standard `city` parser but will need another system to do disambiguation at slot level. You can also make an entity of type `arrival_city` but that will cost in units of transferability.


### 1.4. Filling operators

For utterances where there are various semantic operators used over entities like `4 pm or 9 pm`, slot parsing output becomes structured like `{"op": "or", "args": ["4 pm", "9 pm"]}`. Again, entities form the low level parses here that are understood at a higher level by slot parser.


### 1.5. Repairs and multi turn recovery

Going beyond turns and covering dialog level strategies (like skiplisting), we realize that multi turn slot capture is a thing (and not entity). This would mean even with poor slot capture performance, we _might_ get to better eventual performance.


Note that in all the examples above we could have gone ahead without intermediate entity parsing step but practically that's not a good decision since we already have many low level entity parsers that are transferrable.

## 2. Standard slot-filling metrics

The academic standard is to take slots tagged in BIO format and do span matches to get precision, recall, and f1. Here is the structure from the famous [conlleval](https://github.com/sighsmile/conlleval) script:


```
   Boeing NNP B-NP I-NP
   's POS B-NP B-NP
   747 CD I-NP I-NP
   jetliners NNS I-NP I-NP
   . . O O
   
   Rockwell NNP B-NP I-NP
   said VBD B-VP B-VP
   the DT B-NP B-NP
   agreement NN I-NP I-NP
```

When you run the script, you get per `class` numbers:

```
   processed 961 tokens with 459 phrases; found: 539 phrases; correct: 371.
   accuracy:  84.08%; precision:  68.83%; recall:  80.83%; FB1:  74.35
                ADJP: precision:   0.00%; recall:   0.00%; FB1:   0.00
                ADVP: precision:  45.45%; recall:  62.50%; FB1:  52.63
                  NP: precision:  64.98%; recall:  78.63%; FB1:  71.16
                  PP: precision:  83.18%; recall:  98.89%; FB1:  90.36
                SBAR: precision:  66.67%; recall:  33.33%; FB1:  44.44
                  VP: precision:  69.00%; recall:  79.31%; FB1:  73.80
```

Think a little about this and see why this is not sufficient for us. There are two sides to it, one from ASR to text, other from text to structured parse.

## 3. Issues with just Precision and Recall

Here is the definition of precision and recall (you can derive f1 from this):

$$
\begin{aligned}
\text{precision} &= \frac{tp}{tp + fp} \\
\text{recall} &= \frac{tp}{tp + fn}
\end{aligned}
$$

Now consider each case one by one:


|    | Truth | Prediction |
|-----|-------|------------|
|  tp | $x$   | $x$ |
|  fp | $\phi$ | $x$ |
|  tn | $\phi$ | $\phi$ |
|  fn | $x$   | $\phi$ |


What if the truth is $x$ but the prediction is $y$ and not $\phi$? This implies there is a presence of an equality function that works on non-null values and tells if they are equal. Implicitly, the `conlleval` setup uses a function that exact matches `(span, class)`.

If the `class` is fixed then per `class` F1, precision, and recall work fine. But in our case, where BIO tagging gets hits from both sides—transcription noise and parsing failures—deliberately designed equality function takes the center stage. You can always parametrize the classses but that doesn't work for open sets. You might also notice that the 4 values now don't sum to 100% unless you redefine `class` by breaking open value of entities, but that would mean that `7 pm` and `11 pm` are different classes.

## 4. MM, MF, NF

Here is a more useful table, extending the previous one:


|    | Truth | Prediction |
|-----|-------|------------|
|  tp | $x|y$   | $x|y$ |
|  tp[$\ne$] | $x|y$ | $y|x$ |
|  fp | $\phi$ | $x | y$ |
|  tn | $\phi$ | $\phi$ |
|  fn | $x|y$   | $\phi$ |


Metrics should be helpful in deciding what to do next. Given our context, we found misfires, mismatches, and nofires to be useful. High misfires tells that the ASR or parsers are too trigger happy. High nofires tells that we are missing too much and might need to check if rules or lexicons are lacking. High mismatches tells us, mostly, that we are not able to deal with transcription noise well in parsing. Here are the definitions:

$$
\begin{aligned}
\text{misfires (fpr)} &= \frac{fp}{fp + tn} \\
\text{mismatches} &= \frac{tp[\ne]}{tp + tp[\ne]} \\
\text{nofires (fnr)} &= \frac{fn}{tp + tp[\ne] + fn}
\end{aligned}
$$

Note that mismatches' definition has changed from its last usage.

Since we have been using the previous terminologies for a while, I used them to explain here. But going forwards, let's start using the standard terminologies known in the field. This means the following changes going forward:

1. _Misfires_ becomes False Positive Rate (FPR) or Fall-Out
1. _Nofires_ becomes False Negative Rate (FNR) or Miss Rate

Mismatches should also be renamed to something else like _accuracy_, but that's wrong. Maybe we can keep it like this.

Read more [here](https://en.wikipedia.org/wiki/Precision_and_recall).

## 5. What to Report?

We can still keep a single metric that tells the general state of the specific **slot** parser (entity reporting is internal and can be done with the same tool and processes) while keep the internal breakdown as previously described.

We can try accuracy as the single metric. This would be the following:

$$\text{accuracy} = \frac{tp + tn}{tp + tp[\ne] + fp + tn + fn}$$

But this very non robust and isn't going to be standard across flows since $tn$ will change drastically based on the flow. The impact of $tn$ makes $\text{fpr}$ also non standard and useless. Remember, one slot usually comes in action only during a brief period of the whole conversation.

A good single metric that also covers a missing aspect is **Slot Capture Rate (SCR)** (_new nomenclature_). SCR goes across  turns and tells if at the end of the call (whether dropoff or transferred or resolved) the slot value filled was correct or not. Minimum criteria for counting a call here is to ensure that the user provided the slot value in any turn, with or without bot's initiation.

SCR can be estimated fairly well by tracking FSMs state and using slot tagged data. Similar to intents, we can also come up with sub-flow where the slot was being pursued and define **Slot Retry Rate** as a metric tracking user annoyance. For debugging purposes and developing dialog level strategies, we will also be interested in the `@k` class of metrics which makes sense only for mismatches.

Finally, here is the recommendation for reporting, all based on turn based slot tagging:

- Top level
  1. SCR
  1. Slot Retry Rate
- Turn level
  1. FPR
  1. FNR
  1. Mismatches
- Misc
  1. Mismatches@k

## 6. How to Report?

TODO: Run live
- Do you need range?
- How to tag?
- What about items not in transcription?
- How to sample?
- Which tool to use for reporting?

## 7. Action items

TODO: Run live
- Who owns the process?
- Who builds the tool?